# Introduction

Le but de ce document est d’expliquer une démonstration du théorème de Cantor-Berntein : "S'il existe une injection d'un ensemble $E$ vers un ensemble $F$ et une injection de $F$ vers $E$, alors il existe une bijection de $E$ sur $F$."

Puis de créer un programme génère une bijection à partir des deux injections en s’inspirant de cette démonstration, et enfin de s’en servir sur un exemple. (solution non générale)

# Explication avec les mains (et un humour douteux)

Nomons u la bijection de E sur F et v la bijection de $F$ sur $E$.

In [23]:
# Just add some julia package
using Pkg
Pkg.add("Primes")
Pkg.add("SaferIntegers");

 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]


In [3]:
using Primes
using SaferIntegers

S = NTuple{N,Int} where N
"An injection from S to ℕ (where S is the set of all finite sequences)"
function f(x::S)::Int
    if isempty(x)
        return 0
    end
    res = SafeInt(1)
    for (i, xi) in enumerate(x)
        print(i)
        ti = SafeInt(prime(i))^(xi+1)
        res *= ti
    end
    return res
end

"An basic injection from ℕ to S"
g(x::Int)=(x,)

ginv(x::S)=x[1];

function finv(x::Int)
    if x == 0
        return ()
    end
    if x == 1
        throw(DomainError("ptdr t ki ?"))
    end
    factors = factor(x)
    res = []
    for (i, f) in enumerate(factors)
        if f[1] != prime(i)
            throw(DomainError("finv undefined on $x"))
        end
        push!(res, f[2]-1)
    end
    return Tuple(res)
end

x = (0,3,1,0,2)
println(x)
y = f(x)
println(y)
x = finv(y)
println(x)

(0, 3, 1, 0, 2)
1234537733850
(0, 3, 1, 0, 2)


In [20]:
struct C0
end

struct C
end

"C0 = S-g[ℕ]"
Base.in(x::Tuple, ::Type{C0}) = length(x)!=1

"C = ⋃Cn"
function Base.in(x::Tuple, ::Type{C})
    if x in C0
        return true
    end

    # x∈C1 ⇔ x∈g(f(C0)) ⇔ y∈C0 x=g(f(y)) ⇔ y∈C0 y=f^-1(g^-1(x))
    try
        y=finv(ginv(x))
        if y in C0
            return true
        end
    catch
    end
    # From here x in Cn ⇒ x=2^(k+1) and g(k) in C(n-1)
    
    # g^-1(x)=2^(k)
    k = log2(ginv(x))-1
    if !isinteger(k) || k<0
        return false
    end
    k = Int(k)
    return true
    return g(k) in C
end

(16,) in C

true